<a href="https://colab.research.google.com/github/mmsamiei/world/blob/main/world.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install Faker

In [2]:
from faker import Faker
import random
import datetime
from itertools import filterfalse
from dateutil import relativedelta


In [3]:
fake = Faker()

In [4]:
class Location:

  def __init__(self, map_size=50):
    self.x =  random.uniform(-map_size, map_size)
    self.y = random.uniform(-map_size, map_size)

  def distance(self, other_location):
    dx = other_location.x - self.x
    dy = other_location.y - self.y
    return (dx**2+dy**2)**0.5

In [19]:
class Person:
  
  def __init__(self, world):

    self.world = world
    self.gender = random.choices(['M', 'F'])
    self.first_name = fake.first_name_female() if self.gender == 'F' else fake.first_name_male()
    self.last_name = fake.last_name()
    self.birthday = self.world.date
    self.deathday = None
    self.location = Location()
    self.velocity = 10
    self.energy = 100

  def __str__(self):
    x = round(self.location.x,2)
    y = round(self.location.y,2)
    age = relativedelta.relativedelta(self.world.date, self.birthday)

    return f"{self.first_name} {self.last_name}  at {x}, {y} energy: {self.energy}  age: {age.years, age.months}"
  
  def choose_action(self):
    
    for food in self.world.foods:
      if self.location.distance(food.location) < self.world.distance_can_eat:
        return 'eat', (self, food)

    day_to_death = self.energy / self.world.hunger_penalty
    dx = 0
    dy = 0
    for food in self.world.foods:
      if self.location.distance(food.location)/self.velocity < day_to_death:
        dx = food.location.x - self.location.x
        dy = food.location.y - self.location.y
        length = (dx**2+dy**2)**0.5
        dx_norm = (dx / length) * (self.velocity ** 0.5)
        dy_norm = (dy / length) * (self.velocity ** 0.5)
        if dx**2 + dy**2 < dx_norm**2 + dy_norm**2:
          dir_x, dir_y = dx, dy
        else:
          dir_x, dir_y= dx_norm, dy_norm
        break
    return 'move', (self, dx, dy)

In [20]:
class Food:

  def __init__(self, world):
    self.world = world
    self.location = Location()
    self.produce_date = self.world.date

In [30]:
class World:

  def __init__(self, name):
    self.name = name
    self.persons = []
    self.dead_persons = []
    self.date = datetime.date(2018, 6, 1)
    self.foods = []
    self.hunger_penalty = 10
    self.distance_can_eat = 1
    self.every_day_foods = 1000

  def simulate_day(self):
    self.date += datetime.timedelta(days=1)

    new_person = Person(world=self)
    self.persons.append(new_person)

    for i in range(self.every_day_foods):
      self.foods.append(Food(self))
    self.foods[:] = filterfalse(lambda f: (world.date - f.produce_date).days > 5 , self.foods)

    for person in self.persons[:]:
      self.simulate_person(person)
  

  def simulate_person(self, person):
    person.energy -= self.hunger_penalty
    action, args = person.choose_action()
    
    if action == 'eat':
      food = args[1]
      self.foods.remove(food)
      person.energy = 100
    elif action == 'move':
      dx, dy = args[1], args[2]
      person.location.x += dx
      person.location.y += dy

    if person.energy == 0 :
      self.persons.remove(person)
      self.dead_persons.append(person)
      person.deathday = self.date

  def simulate(self, until=365*20):
    for i in range(until):
      self.simulate_day()
      #print(len(self.foods))
      print("persons ",len(self.persons), len(self.dead_persons))
      


In [31]:
world = World('mahdi_world')
world.simulate()

persons  1 0
persons  2 0
persons  3 0
persons  4 0
persons  5 0
persons  6 0
persons  7 0
persons  8 0
persons  9 0
persons  10 0
persons  11 0
persons  12 0
persons  13 0
persons  14 0
persons  15 0
persons  16 0
persons  17 0
persons  18 0
persons  19 0
persons  20 0
persons  21 0
persons  22 0
persons  23 0
persons  24 0
persons  25 0
persons  26 0
persons  27 0
persons  28 0
persons  29 0
persons  30 0
persons  31 0
persons  32 0
persons  33 0
persons  34 0
persons  35 0
persons  35 1
persons  36 1
persons  37 1
persons  38 1
persons  39 1
persons  40 1
persons  38 4
persons  38 5
persons  37 7
persons  38 7
persons  39 7
persons  40 7
persons  41 7
persons  42 7
persons  43 7
persons  43 8
persons  43 9
persons  44 9
persons  45 9
persons  46 9
persons  46 10
persons  46 11
persons  47 11
persons  48 11
persons  45 15
persons  41 20
persons  40 22
persons  41 22
persons  42 22
persons  43 22
persons  44 22
persons  45 22
persons  46 22
persons  47 22
persons  48 22
persons  49 22

KeyboardInterrupt: ignored

In [32]:
for person in world.persons:
  print(person)

Greg Powell  at -26.7, 10.82 energy: 100  age: (7, 3)
Michael Ewing  at -49.84, -34.25 energy: 90  age: (7, 3)
Stephen Brown  at -26.7, 10.82 energy: 100  age: (7, 3)
Nicholas Miller  at -26.7, 10.82 energy: 100  age: (7, 3)
Daniel Martinez  at -49.84, -34.25 energy: 90  age: (7, 3)
David Palmer  at -26.7, 10.82 energy: 100  age: (1, 9)
Nicholas Spencer  at -49.84, -34.25 energy: 90  age: (1, 9)
Jason Henderson  at -49.84, -34.25 energy: 90  age: (1, 1)
Michael Wright  at -49.84, -34.25 energy: 80  age: (0, 11)
Brandon Romero  at -49.84, -34.25 energy: 80  age: (0, 5)
John Patel  at 20.82, 11.09 energy: 100  age: (0, 5)
Rodney Mcguire  at -18.94, -42.14 energy: 100  age: (0, 4)
Troy Martin  at -49.84, -34.25 energy: 80  age: (0, 4)
Peter Harris  at -49.84, -34.25 energy: 90  age: (0, 3)
Joseph Keller  at -49.84, -34.25 energy: 70  age: (0, 2)
Michael Harper  at -18.97, 41.25 energy: 40  age: (0, 2)
Michael Baker  at -49.84, -34.25 energy: 90  age: (0, 2)
James Williams  at -49.84, -34.

In [25]:
a = datetime.date(2018, 3,3)
b = datetime.date(2013, 6,1)

In [26]:
a-b

datetime.timedelta(days=1736)

In [27]:
a.year - b.year

5